In [1]:
import datetime
import os
import pandas as pd

from src.data_ingestion.db.postgres_db import FinancialDB, PostgresConfig, PostgresDB
from src.data_ingestion.downloader.tiingo_ticker_downloader import TiingoDownloader

COMPUTE_DATE = '2024-08-07'

db_config = PostgresConfig(
    host=os.environ["SB_DDBB_HOST"],
    port=os.environ["SB_DDBB_PORT"],
    database=os.environ["SB_DDBB_DATABASE"],
    user=os.environ["SB_DDBB_USER"],
    password=os.environ["SB_DDBB_PWD"],
)
db = FinancialDB(config=db_config)

tickers = db.get_available_tickers(date=None) # TODO: Change this to top_n tickers, as it is taking so much time

dwn = TiingoDownloader(db=db)

#TODO: Remove this chunk of code as it is required for daily volume/ market cap
downloaded_tickers = pd.read_sql("SELECT distinct(ticker) FROM ticker_prices", db.engine)["ticker"].to_list()
tickers = [i for i in tickers if i not in downloaded_tickers]
##

date = datetime.datetime.now() - datetime.timedelta(days=1)
if COMPUTE_DATE:
    date = datetime.datetime.strptime(COMPUTE_DATE, "%Y-%m-%d")

print(f"Downloading data for date {date}")
print(f"Ticker list lenght {len(tickers)}")

Ticker list lenght 601


In [6]:
def get_top_volume_tickers(top_n: int | None = 100) -> pd.DataFrame:
    limit = ""
    if top_n:
        limit = f"LIMIT {top_n}"

    return pd.read_sql(
        f"""
        SELECT *, Row_number()OVER() as top_n
        FROM (
            SELECT ticker, max(close * volume) as daily_volume FROM ticker_prices WHERE close is not null GROUP BY 1 ORDER BY daily_volume DESC {limit}
        ) a
        """,
        db.engine,
    )

In [34]:
get_top_volume_tickers(10)

,ticker,daily_volume,top_n
0,nvda,4.069557e+10,1
1,qqq,2.388155e+10,2
2,tsla,1.364560e+10,3
3,aapl,1.332701e+10,4
4,meta,9.829887e+09,5
5,smci,8.927984e+09,6
6,msft,8.227940e+09,7
7,amzn,7.879409e+09,8
8,amd,7.035224e+09,9
9,avgo,4.053999e+09,10


In [2]:
FinancialDB(config=db_config).get_top_volume_tickers(10)

['nvda', 'qqq', 'tsla', 'aapl', 'meta', 'smci', 'msft', 'amzn', 'amd', 'avgo']

## Store to volume_tickers database

In [ ]:
REPLACE_TABLE = False

In [7]:
all_tickers_ordered = get_top_volume_tickers(None)
all_tickers_ordered.head(5)

In [10]:
if REPLACE_TABLE:
    all_tickers_ordered[["ticker", "top_n"]].to_sql("volume_tickers", db.engine, if_exists="replace", index=False)

745